The sensordata contains error as numbers:  
For 'HumidityBathroom'  
    - 255 'no connection'  
    - 242 'connection error'
  
For 'VentilationLevel'  
    - 255 'not used'  

For 'Presence':    
    - 255 'no connection'  
    - 242 'connection error'
   

# Imports

In [1]:
import pandas as pd
import numpy as np
import glob

from tqdm import tqdm

%matplotlib inline

Manually change 'Honeywell_season_01' to 'Honeywell_season_01_02' so both honeywell folders are processed.  
Could also make a loop for this, to iterate over the 2 different honeywell.  

In [6]:
# The Honeywell_season_01 data
path = "/datc/opschaler/Honeywell_season_01_02"
sub_dirs = np.array(glob.glob(path + "/*")) # Get location to all subdirectories
dwellings = np.array(list((map(lambda x: x[-11:], sub_dirs))))

In [3]:
# Next thing to do is to iterate over each dwelling and combine all its available data from within the sub directories.

final_dfs = []

In [7]:
for i, sub_dir in enumerate(tqdm(sub_dirs)):
    # Select a sub dir and get all the available files
    sub_dir = sub_dirs[i]
    dwelling = dwellings[i]

    # All the datalog files from the selected dwelling
    files = glob.glob(sub_dir +"/datalogFile_******_201*.csv")
    
    dfs = []
    
    # Read all the files in the selected sub dir, append to a list
    for file in files:
        df = pd.read_csv(file, delimiter=';', parse_dates=['Timestamp'])
        dfs.append(df)
        
    # Concatenate the dfs list into one df. 
    # Original sample rate is 5 minutes.
    data = pd.concat(dfs)
    data = data.rename(columns={'Timestamp': 'datetime'})
    data = data.set_index(['datetime'])

    # Resample to original samplerate so not registered timestamps appear
    data = data.resample('5T').mean()
    
    # Replace the 255 and 242 values with NaNs in the respective columns.
    data[['HumidityBathRoom', 'VentilationLevel', 'Presence']] = data[['HumidityBathRoom', 'VentilationLevel', 'Presence']].replace({255:np.nan, 242:np.nan})
    data['dwelling'] = dwelling
    
    final_dfs.append(data)
    #data.to_csv('/datc/opschaler/honeywell_sensors_per_dwelling_combined/'+dwelling+'.csv', sep='\t', index=True)

100%|██████████| 16/16 [03:22<00:00, 10.89s/it]


In [9]:
final_df = pd.concat(final_dfs)
final_df

/opt/jupyterhub/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4305116 entries, 2017-01-04 00:00:00 to 2017-12-13 23:50:00
Data columns (total 7 columns):
Co2Value            float64
Humidity            float64
HumidityBathRoom    float64
Presence            float64
RoomTemp            float64
VentilationLevel    float64
dwelling            object
dtypes: float64(6), object(1)
memory usage: 262.8+ MB


In [11]:
final_df.to_csv('/datc/opschaler/honeywell_sensors_per_dwelling_combined/honeywell_all_dwellings_combined.csv', sep='\t', index=True)